## Getting the Access Code for Slack

The code below illustrates how we connect to the Slack API to request an authorization token for our app. Remember that we have to register our app with Slack first, and get the `client_id`.

#### Creating a Slack App

* Go to https://api.slack.com/apps and create your app. You will need the "Client ID" and the "Client Secrete" that is created for you.
* Select the "OAuth & Permissions" tab from the left-hand side and add a "Redirect URL" for your app. The redirect URL ensures (for security) that the app can only talk to your own web server. Add `http://<yourIP>:5000/slack` as your redirect URL.
* Select the "Bot Users" tab from the left-hand side and add a bot username for your app.

#### Get the Web Server up and running

See the Python Script `WebServer_Receive_Slack_Authentication.py` that is in this folder. That server should be running (and serve at the `http://<yourIP>:5000/slack` URL, so that it can receive the authentication code for the user.

#### Launch the user authentication process

The code below will create a pop-up window, and will send the user to the Slack authentication page. Slack will verify the identity of the user, and ask the user whether the app is trusted. Once the user logins and grants the permissions, then the Slack server will call the `http://<yourIP>:5000/slack` URL and send the authentication code. 

At that point, out web server (running as a Python Script), will call back Slack saying "hey, I got the code, here is the verification that I am indeed the correct app, here is my `client_secret`. (See the `WebServer_Receive_Slack_Authentication.py` code.)

In [3]:
# This part is required to come up with the pop-up with which Slack will ask for permissions
# Remember to have the Flask webserver up and running (WebServer_Receive_Slack_Authentication.py)
# to receive the code.
from IPython.display import display
from IPython.display import Javascript as JS

slack_url = "https://slack.com/oauth/authorize"
client_id = '132047100118.137144102401'

# remember, the Flask server needs to run
# also, the URL below needs to point to your own server
redirect_uri = 'http://ipython.ipeirotis.com:5000/slack' 
permissions = 'client'


# Send an OAuth request to Slack, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
url = (slack_url + 
    '?response_type=code' + 
    '&client_id='+ client_id + 
    '&scope=' + permissions +
    '&redirect_uri=' + redirect_uri )

# The code below is necessary to pop up the login window
display(JS("window.open('%s')" % url))


<IPython.core.display.Javascript object>

#### Done with the authentication, the authentication data is now stored locally.

After we authorize the app (in a separate browser window), our web server has received from Slack the authentication token, and it was stored in the `slack_secret.json` file. We read it, to ensure that it contains the information that we want.

In [ ]:
import json

# Read the access token from the file
OAUTH_FILE = 'slack_secret.json'
f = open(OAUTH_FILE, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)

auth_info

In [ ]:
# We will just use the library to test that the code that we get back
# allows us to connect to the Slack API
!sudo python3 -m pip install slackclient

In [ ]:
from slackclient import SlackClient
sc = SlackClient(auth_info["access_token"])